In [1]:
import warnings
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [2]:
matches = pd.read_csv('IPL Matches 2008-2021.csv')
balls = pd.read_csv('IPL Ball-by-Ball 2008-2021.csv')
print('Datasets Loaded.')

Datasets Loaded.


In [3]:
#merging matches and balls to make ipl dataset

ipl = balls.merge(matches, on = 'id', how = 'left')
ipl.rename(columns = {'team1' : '1st_team', 'team2' : '2nd_team', 'inning' : 'innings', 'over' : 'overs', 'ball' : 'balls'}, inplace = True)
ipl['date'] = pd.to_datetime(ipl['date'])
ipl = ipl.sort_values(by = ['id', 'innings', 'overs', 'balls'])
ipl.reset_index(inplace = True, drop = True)
print('IPL dataset formed.')

#final score

final_score = pd.DataFrame(ipl.groupby(['id', 'innings'])['total_runs'].sum())
final_score.reset_index(inplace = True)
final_score.rename(columns = {'total_runs' : 'final_score'}, inplace = True)
ipl = ipl.merge(final_score, on = ['id', 'innings'], how = 'left')
print('final_score column completed.')

#total wickets

total_wickets = pd.DataFrame(ipl.groupby(['id', 'innings'])['is_wicket'].sum())
total_wickets.reset_index(inplace = True)
total_wickets.rename(columns = {'is_wicket' : 'total_wickets'}, inplace = True)
ipl = ipl.merge(total_wickets, on = ['id', 'innings'], how = 'left')
print('total_wickets column completed.')

#runs

ipl['runs'] = 0
ipl['runs'][0] = ipl['total_runs'][0]
ipl['runs'][ipl.shape[0]-1] = ipl['final_score'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['id'][i] == ipl['id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['id'][i] != ipl['id'][i+1]:
        ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['id'][i] == ipl['id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
print('runs column completed.')
            
#wickets

ipl['wickets'] = 0
ipl['wickets'][0] = ipl['is_wicket'][0]
ipl['wickets'][ipl.shape[0]-1] = ipl['total_wickets'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['id'][i] == ipl['id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['is_wicket'][i] + ipl['wickets'][i-1]
for i in range(ipl.shape[0]-1):
    if ipl['id'][i] != ipl['id'][i+1]:
        ipl['wickets'][i] = ipl['total_wickets'][i]
for i in range(ipl.shape[0]-1):
    if ipl['id'][i] == ipl['id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['total_wickets'][i]
print('wickets column completed.')
            
#runs last 5 overs

ipl['runs_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['runs_last_5_overs'][i] = ipl['runs'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['id'][i] == ipl['id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['runs_last_5_overs'][i] = ipl['runs'][i] - ipl['runs'][i-30]
print('runs_last_5_overs column completed.')
                
#wickets last 5 overs

ipl['wickets_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['wickets_last_5_overs'][i] = ipl['wickets'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['id'][i] == ipl['id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['wickets_last_5_overs'][i] = ipl['wickets'][i] - ipl['wickets'][i-30]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['id'][i] != ipl['id'][i-1]:
            ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['id'][i] == ipl['id'][i-1]:
            if ipl['innings'][i] != ipl['innings'][i-1]:
                ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]
print('wickets_last_5_overs column completed.')

#cleaning teams

for col in ['batting_team', 'bowling_team']:
    t_list = []
    for i in ipl[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        elif i == 'Kings XI Punjab':
            t_list.append('Punjab Kings')
        else:
            t_list.append(i)
    ipl[col] = t_list
    print(col + ' cleaned.')

#cleaning venues

t_list = []
for i in ipl['venue']:
    if i == 'M.Chinnaswamy Stadium':
        t_list.append('M Chinnaswamy Stadium')
    elif i == 'MA Chidambaram Stadium, Chepauk, Chennai':
        t_list.append('MA Chidambaram Stadium, Chepauk')
    elif i == 'Maharashtra Cricket Association Stadium':
        t_list.append('Subrata Roy Sahara Stadium')
    elif i == 'Punjab Cricket Association IS Bindra Stadium, Mohali':
        t_list.append('Punjab Cricket Association Stadium, Mohali')
    elif i == 'Sardar Patel Stadium, Motera':
        t_list.append('Narendra Modi Stadium, Ahmedabad')
    elif i == 'Wankhede Stadium, Mumbai':
        t_list.append('Wankhede Stadium')
    else:
        t_list.append(i)
ipl['venue'] = t_list
print('venue cleaned.')

IPL dataset formed.
final_score column completed.
total_wickets column completed.
runs column completed.
wickets column completed.
runs_last_5_overs column completed.
wickets_last_5_overs column completed.
batting_team cleaned.
bowling_team cleaned.
venue cleaned.


In [4]:
#rearranging columns

ipl = ipl[['id', 'date', 'city', 'venue', '1st_team', 'neutral_venue', '2nd_team', 'toss_winner', 'toss_decision', 'innings', 'batting_team', 
           'bowling_team', 'batsman', 'non_striker', 'bowler', 'overs', 'balls', 'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary', 
           'is_wicket', 'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type', 'runs', 'wickets', 'runs_last_5_overs', 
           'wickets_last_5_overs', 'winner', 'result', 'result_margin', 'player_of_match', 'eliminator', 'method', 'umpire1', 'umpire2', 
           'final_score', 'total_wickets']]

In [5]:
#Saving file

ipl.to_csv('IPL.csv', index = None)